In [4]:
import pandas as pd
import numpy as np

## GPT3.5

In [8]:
df_600 = pd.read_csv('result1000.csv')

In [9]:
df_600.shape

(1000, 54)

In [27]:
df = df_600.copy()

In [29]:
df_y = pd.read_csv('label_data.csv', encoding = 'utf-8')
df_y = df_y.iloc[:600]

In [30]:
df_y.shape

(600, 42)

In [31]:
df_X = df.reset_index(drop=True)
df_y = df_y.reset_index(drop=True)

data = pd.concat([df_X, df_y], axis=1)
data.head()

,0.2,0.2.1,0.0,0.4,0.2.2,0.7,0.0.1,0.0.2,0.4.1,0.3,...,嗜睡,头晕,腹痛,便血,心悸,螺旋杆菌感染,反流,水肿,胃肠功能紊乱,淋巴结肿大
0,0.1,0.2,0.2,0.2,0.0,0.4,0.2,0.0,0.4,0.3,...,0,0,1,0,0,0,0,0,1,0
1,0.2,0.3,0.0,0.3,0.2,0.2,0.5,0.0,0.4,0.3,...,0,1,1,0,0,0,0,0,0,0
2,0.6,0.6,0.3,0.5,0.0,0.3,0.1,0.5,0.6,0.7,...,0,0,0,1,0,0,0,0,0,0
3,0.5,0.5,0.0,0.5,0.3,0.5,0.0,0.0,0.3,0.7,...,0,0,1,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [32]:
data.shape

(600, 96)

### drop columns

In [35]:
rows_with_all_zeros = df[(df == 0).all(axis=1)]
rows_with_all_zeros

,0.2,0.2.1,0.0,0.4,0.2.2,0.7,0.0.1,0.0.2,0.4.1,0.3,...,0.3.3,0.0.19,0.2.7,0.5.3,0.1.5,0.0.20,0.2.8,0.2.9,0.2.10,0.0.21
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
drop_index = rows_with_all_zeros.index
drop_index

Index([4, 270], dtype='int64')

In [9]:
data = data.drop(index=drop_index)
data.shape

(496, 102)

## TF-IDF

In [41]:
df = pd.read_csv('tfidf_matrix.csv', encoding='utf-8')
df.shape

(13609, 1107)

In [43]:
df_X = df.iloc[:600]

In [44]:
df = pd.read_csv('label_data.csv', encoding = 'utf-8')
df_y = df.iloc[:600]

df_X = df_X.reset_index(drop=True)
df_y = df_y.reset_index(drop=True)

data = pd.concat([df_X, df_y], axis=1)
data.head()

,10,11,12,13,14,15,16,17,18,19,...,嗜睡,头晕,腹痛,便血,心悸,螺旋杆菌感染,反流,水肿,胃肠功能紊乱,淋巴结肿大
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,1,0,0,0,0,0,1,0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074975,0.0,...,0,1,1,0,0,0,0,0,0,0
2,0.076517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,0,1,0,0,0,0,0,0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,1,0,0,0,0,0,0,0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0,0,1,0,0,0,0,0,0,0


In [45]:
data.shape

(600, 1149)

In [30]:
# 平均准确率
def Accuracy(y_true, y_pred):
    count = 0
    for i in range(y_true.shape[0]):
        p = sum(np.logical_and(y_true.iloc[i], y_pred[i]))
        q = sum(np.logical_or(y_true.iloc[i], y_pred[i]))
        count += p / q
    return count / y_true.shape[0]

## Classification 

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# 分割特征和标签
X = data.iloc[:, :-42]  # 假设最后42列是标签
y = data.iloc[:, -2:-1]  # 假设最后42列是标签

# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 由于 SVM 对特征缩放敏感，因此创建一个包含标准化处理的管道
svm_pipeline = make_pipeline(StandardScaler(), OneVsRestClassifier(SVC(probability=True)))

# 训练模型
svm_pipeline.fit(X_train, y_train)

# 进行预测
y_pred = svm_pipeline.predict(X_test)

# 评估模型
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))
# print("Average Accuracy:", Accuracy(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      1.00      0.95       108
           1       0.00      0.00      0.00        12

    accuracy                           0.90       120
   macro avg       0.45      0.50      0.47       120
weighted avg       0.81      0.90      0.85       120

Accuracy Score: 0.9


C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 从合并的 DataFrame 中分割出特征和标签
X = data.iloc[:, :-42]
y = data.iloc[:, -42:]  

# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化随机森林分类器
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
rf_classifier.fit(X_train, y_train)

# 进行预测
y_pred = rf_classifier.predict(X_test)

# 评估模型
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1)))
print("Accuracy Score:", accuracy_score(y_test.values.argmax(axis=1), y_pred.argmax(axis=1)))
# print("Average Accuracy:", Accuracy(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         0
           6       0.75      0.05      0.10        56
           7       0.00      0.00      0.00         9
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00        19
          10       0.00      0.00      0.00        15
          11       0.00      0.00      0.00        25
          12       0.00      0.00      0.00         5
          13       0.00      0.00      0.00         3
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         2
   

C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\26387\.conda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
y = [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_pred = [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [47]:
accuracy_score(y,y_pred)

0.9761904761904762